In [1]:
import pandas as pd
import numpy as np

In [2]:
data= pd.read_csv('explod_preprocessed.csv')

In [3]:
data.drop(['title_id','ans_id','ansdoc','tidoc'], axis=1, inplace=True)

In [4]:
x= data.drop('label',axis=1).values
y=data.label.values

In [6]:
from sklearn.model_selection import train_test_split
xtr,xte,ytr,yte = train_test_split(x,y,test_size=0.2)

In [7]:
import tensorflow.keras
from tensorflow.keras.initializers import HeNormal
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LeakyReLU, PReLU, ELU, Dropout, BatchNormalization
from tensorflow.keras.losses import BinaryCrossentropy

In [8]:
model= Sequential()
model.add(Dense(128,kernel_initializer=HeNormal,input_dim=xtr.shape[1],activation='relu'))
model.add(Dense(256, kernel_initializer=HeNormal,activation='relu'))
model.add(Dense(256, kernel_initializer=HeNormal,activation='relu'))
model.add(Dense(256, kernel_initializer=HeNormal,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(128, kernel_initializer=HeNormal,activation='relu'))
model.add(Dense(1, kernel_initializer=HeNormal,activation='sigmoid'))

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               3456      
_________________________________________________________________
dense_1 (Dense)              (None, 256)               33024     
_________________________________________________________________
dense_2 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_3 (Dense)              (None, 256)               65792     
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 1

In [10]:
model.compile(loss=BinaryCrossentropy(),optimizer='adam',metrics=['accuracy'])

In [11]:
history= model.fit(xtr,ytr, validation_split=0.3, epochs=10)

Epoch 1/10
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
2452/2456 [============================>.] - ETA: 0s - loss: 4.0235 - accuracy: 0.8857 ETA: 0s - loss: 4.2302 - accuWARNING:tensorflow:AutoGraph could not transform <function Model.make_test_function.<locals>.test_function at 0x000002676761B558> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux,

In [12]:
ypred= model.predict_proba(xte)

Instructions for updating:
Please use `model.predict()` instead.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [13]:
newtest= pd.read_csv('test_fuzzy.csv')

In [14]:
newtest_tit= list(newtest['title_id'])
newtestcandi=list(newtest['ans_id'])

In [15]:
newtest.drop(['title_id','ans_id','ansdoc','tidoc'], axis=1, inplace=True)

In [16]:
ypred=model.predict_proba(newtest)

In [17]:
ypred=ypred.flatten()

In [18]:
len(ypred)

35080

In [19]:
resd= {
    'title_id':newtest_tit,
    'ans_id':newtestcandi,
    'prob':ypred
}

In [20]:
res= pd.DataFrame(resd)

In [21]:
res3= res.sort_values(['title_id', 'prob'], ascending=[True, False]).groupby('title_id', sort=False)['ans_id'].agg(list).reset_index()